### Data Extraction

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
import time

In [3]:
df = pd.read_csv("/home/anuttar/Desktop/Assignments/IE647_Assignment3/cutstock.csv")
df = df.rename(columns={"#width": "width"})
print("Total Number of Orders: ", len(df))
df = df[:10]
df.head()

Total Number of Orders:  61


,width,demand
0,1000,706
1,980,665
2,940,1189
3,900,1044
4,860,914


Only <b> 10 samples </b> are used to obtain the solution. Using more (or all) samples took too much time

### Probelm Modelling

#### Variables Used
$W$ : Width of a large roll <br>
$m$ : Number of orders <br>
$d_{i}$ : Demand of $i^{th}$ order <br>
$w_{i}$ : Width of $i^{th}$ order <br>
$x_{i,j}$ : Number of pieces of width $w_{j}$ cut from $i^{th}$ large roll <br>
$y_{i}$ : Boolean variable that denotes whether ith large roll is used (1) or not (0) <br>
$p$ : Upper bound on number of large rolls $ (p = \sum_{j=1}^{m} d_j )$  <br>

#### Cutstock Model
$
\text{min} \sum_{j=1}^{p} y_j
$

Subject to:

$
\sum_{j=1}^{m} x_{ij} w_{j} \leq W y_i, \quad i = 1, \dots, p
$

$
\sum_{i=1}^{p} x_{ij} \geq d_j, \quad j = 1, \dots, m
$

$
y_i \in \{0,1\}, \quad i = 1, \dots, p
$

$
x_{ij} \in \mathbb{Z}_+, \quad i = 1, \dots, p, \quad j = 1, \dots, m.
$

(Referenced from CCZ)

### Solution

Solver Used: CBC

#### Using complete Integer Programming Formulation

In [9]:
start_time = time.time()

#Initialisation
m = len(df["demand"])
p = np.sum(df["demand"])
w = np.array(df["width"])
d = np.array(df["demand"])
W = 1030

#Define Model
model = pyo.ConcreteModel()

#Define Variables
model.y = pyo.Var(range(p), within=pyo.Binary)
model.X = pyo.Var(range(p), range(m), within=pyo.NonNegativeIntegers)

#Define Objective Function
model.obj = pyo.Objective(expr = sum(model.y[j] for j in range(p)), sense = pyo.minimize)

#Adding Constraints
model.constraints = pyo.ConstraintList()
z1 = {i: sum(model.X[i, j] * w[j] for j in range(m)) for i in range(p)}
for i in range(p):
    model.constraints.add(z1[i] <= W * model.y[i])

z2 = {j: sum(model.X[i, j] for i in range(p)) for j in range(m)}
for i in range(m):
    model.constraints.add(z2[i] == d[i])
    
# Select solver
solver = pyo.SolverFactory('cbc')

# Solve the problem
result = solver.solve(model)

# Display results
print('Status:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)
print('Optimal Objective:', pyo.value(model.obj))

end_time = time.time()
print("Time taken: ", end_time - start_time, " seconds")

Status: ok
Termination Condition: optimal
Optimal Objective: 19822.0
Time taken:  3164.8934886455536  seconds


#### Using LP Relaxation

In [6]:
import pyomo.environ as pyo
import time

start_time = time.time()

#Initialisation
m = len(df["demand"])
p = np.sum(df["demand"])
w = np.array(df["width"])
d = np.array(df["demand"])
W = 1030

#Define Model
model = pyo.ConcreteModel()

#Define Variables
model.y = pyo.Var(range(p), within=pyo.NonNegativeReals)
model.X = pyo.Var(range(p), range(m), within=pyo.NonNegativeReals)

#Define Objective Function
model.obj = pyo.Objective(expr = sum(model.y[j] for j in range(p)), sense = pyo.minimize)

#Adding Constraints
model.constraints = pyo.ConstraintList()
z1 = {i: sum(model.X[i, j] * w[j] for j in range(m)) for i in range(p)}
for i in range(p):
    model.constraints.add(z1[i] <= W * model.y[i])

z2 = {j: sum(model.X[i, j] for i in range(p)) for j in range(m)}
for i in range(m):
    model.constraints.add(z2[i] == d[i])
    
# Select solver
solver = pyo.SolverFactory('ipopt')

# Solve the problem
result = solver.solve(model)

# Display results
print('Status:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)
print('Optimal Objective:', pyo.value(model.obj))

end_time = time.time()
print("Time taken: ", end_time - start_time, "seconds")

Status: ok
Termination Condition: optimal
Optimal Objective: 15183.514612497396
Time taken:  3.9636390209198 seconds
